In [ ]:
import json
import re

In [ ]:
def write_json(contenu,chemin):
     '''  cette fonction permet d'écrire des fichiers json
    paramètre: -chemin
                - contenu du fichier'''
    with open(chemin,"w",encoding="utf-8") as f:
        json.dump(contenu,f,indent=4)

def open_json(chemin):
    '''  cette fonction permet d'ouvrir des fichiers json
    paramètre: chemin'''
    with open(chemin, encoding="utf-8") as f:
        fichier= json.load(f)
    return fichier

def find_figement(similarities_and_sent,seuil):
    ''' Cette fonction permet de collecter touts les figements  à partir du moment où leurs mesures de similarité sont supérieur au seuil
    paramètre: - liste contenant les mesures de similarité et les phrases
                - seuil'''
    figements=[]

    for item in similarities_and_sent:
        similarities,tweet= item
        if all(similarities[key][0]>seuil for key in ['TOK', 'LEM', 'POS', 'meanLayer']):
         figements.append(tweet)
         figements= list(set(figements))
    return figements

def find_substitution_force(similarities_and_sent,expression_reg):
    ''' Cette fonction permet de trouver toutes les substitions au sein du figement que la force soit avec toi
    paramètre: - liste contenant les mesures de similarité et les phrases
               - expression régulière pour trouver les substitutions'''
    substitution=[]
    regex= re.compile(expression_reg,re.IGNORECASE)
    for simi, sent in similarities_and_sent:
        match= regex.search(sent)
        if match:
            substitution.append(match.group(1)) #il y tjrs qlqs figements
    substitution=[no_fige for no_fige in substitution if "toi" not in no_fige] #parfait
    return list(set(substitution))

def find_substitution_travail(similarities_and_sent,expression_reg):
    ''' Cette fonction permet de trouver toutes les substitions au sein du figement travailler plus pour gagner plus
    paramètre: - liste contenant les mesures de similarité et les phrases
               - expression régulière pour trouver les substitutions'''
    substitution=[]
    regex= re.compile(expression_reg,re.IGNORECASE)
    for simi, sent in similarities_and_sent:
        match= regex.search(sent)
        if match:
            substitution.append(match.group(0))
    return list(set(substitution))

def find_insertion(similarities_and_sent,expression_reg):
   ''' Cette fonction permet de trouver toutes les défigements de type insertion
    paramètre: - liste contenant les mesures de similarité et les phrases
               - expression régulière pour trouver les insertions'''
    insert=[]
    regex= re.compile(expression_reg, re.IGNORECASE)
    for simi, sent in similarities_and_sent:
        match= regex.search(sent)
        if match:
            insert.append(match.group(0))

    return list(set(insert))

In [ ]:
#MAIN
la_force= open_json("Que_la_force_soit_avec_toi.json")
travail= open_json("Travailler_plus_pour_gagner_plus.json")

#CORPUS QUE LA FORCE SOIT AVEC TOI
#recuperer les mesures de similarité et les tweet dans une liste
similarities_and_sent_force=[]

for elt in la_force:
   similarities_and_sent_force.append([elt["similarities"],elt["sent"]])


#collecter les figements grâce aux mesures de similarité
seuil= 0.60
figements_force= find_figement(similarities_and_sent_force, seuil)

# trouver toutes les substitutions de type que la force soit avec X (au lieu de toi)
substitutions_force= find_substitution_force(similarities_and_sent_force,r"(que la force soit avec (?!toi\b)[\w@\'’\- ]+)")

#trouver les insertion
insertion_force= find_insertion(similarities_and_sent_force,r"que la force \w{1,6} soit avec toi")


#enregistrer d les resulats dans un fichier json
resultat= {"figement":figements_force, "substitution":substitutions_force, "insertion": insertion_force}


write_json(resultat,"resulat_que_la_force_soit_avec_toi.json")



# CORPUS TRAVAILLER PLUS POUR GAGNER PLUS
similarities_and_sent_travail=[]
seuil2= 0.70
for elt in travail:
   similarities_and_sent_travail.append([elt["similarities"],elt["sent"]])
figements_travail= find_figement(similarities_and_sent_travail, seuil2)

substitution_travail= find_substitution_travail(similarities_and_sent_travail,r"travailler \w{5} pour gagner \w{5,10}")

insertion_travail=find_insertion(similarities_and_sent_travail,r"travailler \w{1,6} plus pour gagner \w{1,6} plus" )

resultat2= resultat= {"figement":figements_travail, "substitution":substitution_travail, "insertion": insertion_travail}

write_json(resultat2, "resultat_travailler_plus_pr_gagner_plus.json")
